In [41]:
import pandas as pd
import numpy as np
import datetime as dt
from datetime import datetime
import requests as rq
import io
import pytz

In [42]:
def short_format(num):
    if num > -10000 and num < 10000:
        return '{:,.4g}'.format(num)
    num = float('{:.3g}'.format(num))
    magnitude = 0
    while abs(num) >= 1000:
        magnitude += 1
        num /= 1000.0
    return '{}{}'.format('{:f}'.format(num).rstrip('0').rstrip('.'), ['', 'K', 'M', 'B', 'T'][magnitude])

In [43]:
def get_us_data(date=None, field=None):
    """
    Returns a JSON object with the following notable fields:
    positive, negative, death, total (number of tests), recovered
    hospitalizedCurrently, inIcuCurrently, onVentilatorCurrently
    deathIncrease, hospitalizedIncrease, positiveIncrease, negativeIncrease, totalTestResultsIncrease
    """
    j = rq.get("https://covidtracking.com/api/v1/us/{}.json".format(date.strftime('%Y%m%d') if date else "current")).json()
    j = j if date else j[0]
    return j[field] if field else j

In [44]:
def get_state_data(state, date=None, field=None):
  """
  Returns a JSON object with the following notable fields:
    positive, negative, death, total (number of tests), recovered
    hospitalizedCurrently, inIcuCurrently, onVentilatorCurrently
    deathIncrease, hospitalizedIncrease, positiveIncrease, negativeIncrease, totalTestResultsIncrease
  """
  j = rq.get("https://covidtracking.com/api/v1/states/{}/{}.json".format(state, date.strftime('%Y%m%d') if date else "current")).json()
  return j[field] if field else j

In [45]:
def percent_change(old, new):
  return (new - old) / old * 100

In [46]:
def process_data(imported):
    nyc = pd.DataFrame()
    nyc['Deaths'] = imported['BK_DEATH_COUNT']
    nyc['Cases'] = imported['BK_CASE_COUNT']
    nyc['Total Deaths'] = np.cumsum(nyc['Deaths'])
    nyc['Date'] = imported['date_of_interest']
    
    average = list(np.zeros(4)) + list(np.ones(2)*0.5)
    for i in range(7,len(nyc)+1):
        average += [np.mean(nyc['Cases'][i-7:i])]

    nyc['7 day average'] = average
    
    return nyc

In [47]:
def get_updated_data(nyc, di):
    last_row = nyc.tail(1).iloc[0]
    d_str = datetime.strptime(last_row['Date'], '%m/%d/%Y').strftime('%-m/%-d')
    d_today_str = datetime.now(pytz.timezone('US/Pacific')).strftime('%-m/%-d')
    return {
        "smart_tiles": [
            {
                "figure": short_format(last_row['Deaths']),
                "subheader": "On {}".format(d_str)
            },
            {
                "figure": short_format(last_row['Total Deaths']),
                "subheader": "As of {}".format(d_today_str)
            },
            {
                "figure": short_format(get_us_data(field='death')),
                "subheader": "As of {}".format(d_today_str)
            }
        ]
    }


In [48]:
CSV_URL = "https://raw.githubusercontent.com/nychealth/coronavirus-data/master/trends/data-by-day.csv"
df = pd.read_csv(io.StringIO(rq.get(CSV_URL).text))
df

,date_of_interest,CASE_COUNT,PROBABLE_CASE_COUNT,HOSPITALIZED_COUNT,DEATH_COUNT,PROBABLE_DEATH_COUNT,CASE_COUNT_7DAY_AVG,ALL_CASE_COUNT_7DAY_AVG,HOSP_COUNT_7DAY_AVG,DEATH_COUNT_7DAY_AVG,...,QN_CASE_COUNT_7DAY_AVG,QN_HOSPITALIZED_COUNT_7DAY_AVG,QN_DEATH_COUNT_7DAY_AVG,SI_CASE_COUNT,SI_HOSPITALIZED_COUNT,SI_DEATH_COUNT,SI_CASE_COUNT_7DAY_AVG,SI_HOSPITALIZED_COUNT_7DAY_AVG,SI_DEATH_COUNT_7DAY_AVG,INCOMPLETE
0,02/29/2020,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,03/01/2020,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,03/02/2020,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,03/03/2020,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,03/04/2020,5,0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277,12/02/2020,2817,914,150,21,3,2122,2905,146,14,...,573,43,3,370,20,3,271,18,4,6000
278,12/03/2020,2598,891,186,13,5,2386,3235,153,15,...,647,45,3,309,35,3,303,21,4,6000
279,12/04/2020,2405,763,167,16,8,2416,3252,157,16,...,649,47,4,282,21,3,305,20,4,6000
280,12/05/2020,1276,629,133,15,9,2351,3185,160,16,...,638,47,4,140,16,1,296,21,4,6000


In [49]:
clean_df = process_data(df)
clean_df

,Deaths,Cases,Total Deaths,Date,7 day average
0,0,0,0,02/29/2020,0.000000
1,0,0,0,03/01/2020,0.000000
2,0,0,0,03/02/2020,0.000000
3,0,0,0,03/03/2020,0.000000
4,0,1,0,03/04/2020,0.500000
...,...,...,...,...,...
277,8,757,5860,12/02/2020,595.000000
278,3,738,5863,12/03/2020,666.857143
279,4,646,5867,12/04/2020,676.285714
280,6,339,5873,12/05/2020,662.428571


In [50]:
get_updated_data(clean_df, None)

{'smart_tiles': [{'figure': '1', 'subheader': 'On 12/6'},
  {'figure': '5,874', 'subheader': 'As of 12/9'},
  {'figure': '280K', 'subheader': 'As of 12/9'}]}